In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
sys.path.append('../')

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import svm

from loglizer.models import PCA, IsolationForest, LogClustering, OneClassSVM
from loglizer import dataloader, preprocessing
from loglizer.utils import metrics

D:\Anaconda\envs\LogBert\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\LogBert\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\Anaconda\envs\LogBert\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
ouput_dir = "../output/no_shuffle_hdfs_325/"
(x_train, y_train), (x_test, y_test) = dataloader.load_data(data_dir=ouput_dir)
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train)
x_test = feature_extractor.transform(x_test)

E:\logbert-main\loglizer\dataloader.py:286: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train).reshape(-1,1)
E:\logbert-main\loglizer\dataloader.py:292: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_normal = np.array(test_normal).reshape(-1,1)
E:\logbert-main\loglizer\dataloader.py:298: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when 

Train normal size: 165180
Train abnormal size: 2667
Test normal size: 280863
Test abnormal size: 4001
====== Transformed train data summary ======
Train data shape: 167847-by-45

====== Transformed test data summary ======
Test data shape: 284864-by-45



In [6]:
%%time
print("="*20 + " Model: PCA " + "="*20)
for th in np.arange(1):
    print("theshold", th)
    model = PCA(n_components=0.8, threshold=1, c_alpha = 1.9600)
    model.fit(x_train)
    print('Train validation:')
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Test validation:')
    precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: PCA ====================
theshold 0
====== Model summary ======
n_components: 1
Project matrix shape: 45-by-45
SPE threshold: 1

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 2667, FP: 163006, TN: 2174, FN: 0
Precision: 1.610%, recall: 100.000%, F1-measure: 3.169%

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 4001, FP: 280863, TN: 0, FN: 0
Precision: 1.405%, recall: 100.000%, F1-measure: 2.770%

CPU times: total: 766 ms
Wall time: 1.58 s


In [5]:
%%time
print("="*20 + " Model: IsolationForest " + "="*20)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=19)
model.fit(x_train)
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: IsolationForest ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 2474, FP: 13333, TN: 151847, FN: 193
Precision: 15.651, recall: 92.763, F1-measure: 26.784

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 3707, FP: 7205, TN: 273658, FN: 294
Precision: 33.972, recall: 92.652, F1-measure: 49.715

CPU times: total: 18.4 s
Wall time: 19.6 s


In [ ]:
%%time
print("="*20 + " Model: one class SVM " + "="*20)
model = OneClassSVM(kernel='rbf')
model.fit(x_train, y_train)

print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: one class SVM ====================
====== Model summary ======


In [14]:
# %%time
# print("="*20 + " Model: one class SVM " + "="*20)

# nus = [0.001, 0.01, 0.1, 1]
# gammas = [0.001, 0.01, 0.1, 1]
# tuned_parameters = {'kernel' : ['rbf','poly','linear','sigmoid'], 'gamma' : gammas, 'nu': nus}

# ocsvm = svm.OneClassSVM()
# model = GridSearchCV(ocsvm, tuned_parameters, cv=5, scoring="f1_micro")

# model.fit(x_train, y_train.astype(int))

# # print('Train validation:')
# # precision, recall, f1 = model.predict(x_train, y_train.astype(int))
# # print('Test validation:')
# # precision, recall, f1 = model.predict(x_test, y_test.astype(int))

==================== Model: one class SVM ====================
CPU times: user 2min 50s, sys: 3.56 s, total: 2min 54s
Wall time: 2min 54s


GridSearchCV(cv=5, estimator=OneClassSVM(),
             param_grid={'gamma': [0.001, 0.01, 0.1, 1],
                         'kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
                         'nu': [0.001, 0.01, 0.1, 1]},
             scoring='f1_micro')

In [15]:
# print('Train validation:')
# y_eval = model.predict(x_train)
# precision, recall, f1 = metrics(y_eval, y_train)
# print('Precision: {:.3f}, recall: {:.3f}, F1-measure: {:.3f}\n'.format(precision, recall, f1))
    
# print('Test validation:')
# y_pred = model.predict(x_test)
# precision, recall, f1 = metrics(y_pred, y_test)
# print('Precision: {:.3f}, recall: {:.3f}, F1-measure: {:.3f}\n'.format(precision, recall, f1))


Train validation:
Confusion Matrix: TP: 1543, FP: 5000, TN: 0, FN: 957
Precision: 23.582, recall: 61.720, F1-measure: 34.126

Test validation:
Confusion Matrix: TP: 9114, FP: 553223, TN: 0, FN: 5224
Precision: 1.621, recall: 63.565, F1-measure: 3.161



In [4]:
%%time
print("="*20 + " Model: LogClustering " + "="*20)
max_dist = 0.3  # the threshold to stop the clustering process
anomaly_threshold = 0.3  # the threshold for anomaly detection
model = LogClustering(max_dist=max_dist, anomaly_threshold=anomaly_threshold)
model.fit(x_train[y_train == 0, :])  # Use only normal samples for training
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: LogClustering ====================
====== Model summary ======
Starting offline clustering...
Processed 1000 instances.
Found 4 clusters offline.

Starting online clustering...
Processed 2000 instances.
Processed 4000 instances.
Processed 6000 instances.
Processed 8000 instances.
Processed 10000 instances.
Processed 12000 instances.
Processed 14000 instances.
Processed 16000 instances.
Processed 18000 instances.
Processed 20000 instances.
Processed 22000 instances.
Processed 24000 instances.
Processed 26000 instances.
Processed 28000 instances.
Processed 30000 instances.
Processed 32000 instances.
Processed 34000 instances.
Processed 36000 instances.
Processed 38000 instances.
Processed 40000 instances.
Processed 42000 instances.
Processed 44000 instances.
Processed 46000 instances.
Processed 48000 instances.
Processed 50000 instances.
Processed 52000 instances.
Processed 54000 instances.
Processed 56000 instances.
Processed 58000 instances.
Processed 60000 